# 06 Relationships between customer behaviors

In [1]:
import sqlalchemy
import pandas as pd

# Make a sql connection with sqlalchmey
conn_string = "postgresql://postgres-db/churn?user=postgres&password=password" 
engine = sqlalchemy.create_engine(conn_string)
conn = engine.connect()

# Query with Pandas, e.g. list all tables
tables = pd.read_sql_query("SELECT * FROM information_schema.tables;", conn)
tables.head(3)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,churn,socialnet7,event,BASE TABLE,None,None,None,None,None,YES,NO,None
1,churn,socialnet7,subscription,BASE TABLE,None,None,None,None,None,YES,NO,None
2,churn,socialnet7,event_type,BASE TABLE,None,None,None,None,None,YES,NO,None


In [4]:
query = """\
WITH RECURSIVE active_period_params AS
(
    SELECT
        interval '%gap_interval' AS allowed_gap,
        '%to_yyyy-mm-dd'::date AS calc_date
),
active AS
(
    SELECT
  DISTINCT
        account_id,
        min(start_date) AS start_date
      FROM subscription
INNER JOIN active_period_params
        ON start_date <= calc_date
       AND (end_date > calc_date OR end_date IS null)
  GROUP BY account_id

    UNION

    SELECT
        s.account_id,
        s.start_date
      FROM subscription s
CROSS JOIN active_period_params
INNER JOIN active e
        ON s.account_id=e.account_id
       AND s.start_date < e.start_date
       AND s.end_date >= (e.start_date-allowed_gap)::date

)
INSERT INTO active_period (account_id, start_date, churn_date)
     SELECT
        account_id,
        min(start_date) AS start_date,
        NULL::date as churn_date
       FROM active
   GROUP BY account_id, churn_date

"""
result = pd.read_sql_query(query, conn)
result.head()

TypeError: dict is not a sequence

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

def metric_pair_plot(data_set_path, metric1='',metric2=''):

    churn_data = pd.read_csv(data_set_path,index_col=[0,1])

    met1_series = churn_data[metric1]
    met2_series = churn_data[metric2]

    corr = met1_series.corr(met2_series)

    plt.scatter(met1_series, met2_series, marker='.')

    plt.xlabel(metric1)
    plt.ylabel(metric2)
    plt.title('Correlation = %.2f' % corr)
    plt.tight_layout()
    plt.grid()

    save_name = data_set_path.replace('.csv', '_' + metric1 + '_vs_' + metric2 + '.png')
    plt.savefig(save_name )
    print('Saving plot to %s' % save_name)
    plt.close()
